In [ ]:
%pip install langchain neo4j python-dotenv
%pip install -U langchain_community
%pip install pypdf tiktoken
%pip install -U langchain-ollama
%pip install sentence-transformers
%pip install hf_xet
%pip install -U langchain-neo4j
%pip install -U langchain-huggingface

In [ ]:
from langchain_neo4j import Neo4jGraph
from dotenv import load_dotenv
import os

load_dotenv(override=True)

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

# Test the connection
print(graph.query("MATCH (n) RETURN count(n) AS count"))


[{'count': 43}]


In [27]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("books/ncert_class_x_science_chapter_1.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

print(len(docs))

43


In [28]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2" 
)


In [29]:
from langchain_neo4j import Neo4jVector

# Store documents in Neo4j as nodes with vector embeddings
vector_store = Neo4jVector.from_documents(
    docs,
    embedding=embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="document_chunks",
    node_label="Chunk",
    text_node_property="text",
)

In [30]:
cypher_schema = """
Node labels:
- Chunk: Represents a chunk of a document. Properties: text

No relationships between nodes.
"""

In [37]:
from langchain_ollama.llms import OllamaLLM
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain

# Connect to your graph
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="student1")

# Create LLM
llm = OllamaLLM(model="gemma3:1b")

# Create the chain
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,  
    verbose=True,
    cypher_schema=cypher_schema,
    allow_dangerous_requests=True
)

# Run query
result = chain.invoke({"query": "What do you about chemical reactions?"})
print(result)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:Chunk)
MATCH (n:Chunk)-[:HAS]->(r:Relationship)-[k:Chemical]->(c:Chemical)
RETURN c.id AS chemical_id



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS)} {position: line: 3, column: 19, offset: 41} for query: 'cypher\nMATCH (n:Chunk)\nMATCH (n:Chunk)-[:HAS]->(r:Relationship)-[k:Chemical]->(c:Chemical)\nRETURN c.id AS chemical_id\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that 

Full Context:
[]

> Finished chain.
{'query': 'What do you about chemical reactions?', 'result': "I don't know about chemical reactions."}


In [32]:
from IPython.display import display, Markdown

display(Markdown(result['result']))


I don't know the answer.

In [33]:
res = chain.invoke({"query": "How do you define a chemical reaction?"})
display(Markdown(res["result"]))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:Chunk)-[*]->(r:Relationship)
WHERE n.text = "chemical reaction"
RETURN r.id



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Relationship)} {position: line: 2, column: 25, offset: 31} for query: 'cypher\nMATCH (n:Chunk)-[*]->(r:Relationship)\nWHERE n.text = "chemical reaction"\nRETURN r.id\n'


Full Context:
[]

> Finished chain.


I don’t have enough information to answer this question.

In [34]:
res = chain.invoke({"query": "How do you balance a chemical reaction?"})
display(Markdown(res["result"]))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (n:Chunk)
MATCH (n:Chunk)-[:NOTE]->(r:Note)
CREATE (r)-[:BALANCE]->(n)

Full Context:
[]

> Finished chain.


I’m sorry, I don’t have enough information to answer your question.

In [35]:
res = chain.invoke({"query": "Types of chemical reactions"})
display(Markdown(res["result"]))



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: REACTION)} {position: line: 2, column: 13, offset: 19} for query: 'cypher\nMATCH (n)-[:REACTION]->(r)\nRETURN r.type\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missi

Generated Cypher:
cypher
MATCH (n)-[:REACTION]->(r)
RETURN r.type

Full Context:
[]

> Finished chain.


I don't know the answer to that question.